# This notebook is used to fine-tune flan-t5-large model on k8s question-command pairs gathered from k8s documentation

In [ ]:
!pip install transformers[torch] tokenizers datasets evaluate peft accelerate bitsandbytes rouge_score sentencepiece tensorboard py7zr loralib sklearn huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nltk
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

## Loading the dataset

In [ ]:
dataset = load_dataset("ComponentSoft/k8s-kubectl-35k")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/34884 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset['train'].train_test_split(test_size=0.1, shuffle=False)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

In [ ]:
def preprocess_function(examples):
    """tokenize the text, and set the labels"""

    inputs = [doc for doc in examples['question']]

    inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

    labels = tokenizer(text_target=examples['command'], max_length=512, truncation=True, padding='max_length', return_tensors='pt')

    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'labels': labels.input_ids,
    }


In [ ]:
# Map the preprocessing function across our dataset
tokenized_dataset_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_dataset_eval = dataset['test'].map(preprocess_function, batched=True)

## Set up model for LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

lora_config = LoraConfig(
  r=16,
  lora_alpha=32,
  target_modules=["q", "v"],
  lora_dropout=0.05,
  bias="none",
  task_type=TaskType.SEQ_2_SEQ_LM
)

model = prepare_model_for_int8_training(model)

model = get_peft_model(model, lora_config)

In [ ]:
from transformers import DataCollatorForSeq2Seq

label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

## Set up training args and start training

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="flan-t5-large-k8s-question-code"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	  auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [ ]:
trainer.train()

Step,Training Loss
500,0.122900
1000,0.094800
1500,0.083500
2000,0.082300
2500,0.067700
3000,0.893400
3500,0.113800
4000,0.080400
4500,0.065600
5000,0.059300


TrainOutput(global_step=39245, training_loss=0.05676162180011782, metrics={'train_runtime': 26387.9365, 'train_samples_per_second': 5.949, 'train_steps_per_second': 1.487, 'total_flos': 3.640663824924672e+17, 'train_loss': 0.05676162180011782, 'epoch': 5.0})

In [ ]:
model.push_to_hub('ComponentSoft/flan-t5-large-k8s-question-code')

adapter_model.bin:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ComponentSoft/flan-t5-xl-k8s-question-code/commit/0c1b88c6830dd57fff2eb1cf9402c7971b6b264b', commit_message='Upload model', commit_description='', oid='0c1b88c6830dd57fff2eb1cf9402c7971b6b264b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub('ComponentSoft/flan-t5-large-k8s-question-code')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ComponentSoft/flan-t5-xl-k8s-question-code/commit/290160a0fae6e3aa608f88c2e1a45678184f4377', commit_message='Upload tokenizer', commit_description='', oid='290160a0fae6e3aa608f88c2e1a45678184f4377', pr_url=None, pr_revision=None, pr_num=None)